# notebook/02_data_transformation_to_bronze

In [1]:
import os
import sys 
import logging

# Caminho para a raiz do seu projeto
project_root = os.path.abspath("..")  # Sobe um nível de pasta a partir de 'notebooks'

# Adiciona o caminho ao sys.path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils import functions as F
from config.configs import *
from config.spark_session import create_spark_session

## Spark Session

In [2]:
spark = create_spark_session(HOST_ADDRESS, MINIO_ACCESS_KEY, MINIO_SECRET_KEY)

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-13e0a7c9-ae2c-470b-be8f-b472ed0b0ec2;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.postgresql#postgresql;42.6.0 in central
	found org.checkerframework#checker-qual;3.31.0 in central
:: resolution report :: resolve 130ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.checkerframework#checker-qual;3.31.0 from central in [default]
	org.postgresql#postgresql;42.6.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   5   |   0   |   0   |   0   ||   5   |   0   |
	-----------

## Log configs

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Starting convertions from Raw(CSV) to Bronze(Delta) COVID19...")

2024-11-18 11:27:28,438 - INFO - Starting convertions from Raw(CSV) to Bronze(Delta) COVID19...


## Path configs

In [4]:
input_file_name = file_name['0']
input_prefix_layer_name = prefix_layer_name['0']   # raw layer
table_input_name = lake_path['raw']

output_prefix_layer_name = prefix_layer_name['1']   # bronze layer
storage_output = lake_path['bronze']
output_tabela_name = tabela_name['0']  # nivel agregação

output_full_name = f"{storage_output}{output_prefix_layer_name}{output_tabela_name}"

## Process

In [5]:
# Carregar o arquivo CSV e garantir que a primeira linha seja tratada como o cabeçalho
df_input_data = spark.read.format("csv").option("header", "true").load(f'{project_root}/{table_input_name}/{input_file_name}')

# Adicionando metadados (se necessário)
df_with_update_date = F.add_metadata(df_input_data)

# Escrevendo o DataFrame em formato Delta e particionando pela coluna "aggregation_level"
df_with_update_date.write.format("delta").mode("overwrite").save(f'{output_full_name}')

logging.info("Processing completed successfully!")

24/11/18 11:27:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/11/18 11:27:30 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
2024-11-18 11:27:38,181 - INFO - Processing completed successfully!             
